In [ ]:
 """
 Данный файл обрабатывает данные для постов пользователя с помощью нейросети. 
 Идея - закодировать посты в виде эмбедингов и использовать на них метод понижения пространства что бы использовать как фичи
 для обучения ML модели. Использую лингвистическю модель DistilBertModel, т.к. она легче Bert и Roberta,
 но не сильно уступает им в качестве. В качестве метода понижения пространства - PCA
 """

In [5]:
#забираю данные
import pandas as pd
posts_info = pd.read_csv('post.csv')

In [2]:

from transformers import DistilBertModel  # https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertModel


def get_model(model_name):
    """
    Функция возвращает токенайзер и предобученую модель по ключу 'distilbert'
    """
    assert model_name in ['distilbert']

    checkpoint_names = {'distilbert': 'distilbert-base-cased'  # https://huggingface.co/distilbert-base-cased}

    model_classes = {'distilbert': DistilBertModel}

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

In [3]:
tokenizer, model = get_model('distilbert')

In [6]:
### Напишем класс датасет для постов

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding


class PostDataset(Dataset):
    """
    Класс обрабатывает полученый текст с помощью токнайзера 'distilbert'
    для использования в этой моделе    
    """
    def __init__(self, texts, tokenizer):
        super().__init__()

        self.texts = tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True,
            padding=True
        )
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        return {'input_ids': self.texts['input_ids'][idx], 'attention_mask': self.texts['attention_mask'][idx]}

    def __len__(self):
        return len(self.texts['input_ids'])
    
#создадим экземпляр класса для датасета    
dataset = PostDataset(posts_info['text'].values.tolist(), tokenizer)
#укажем способ сбора батчей с паддингом
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#сформируем даталоадер для модели
loader = DataLoader(dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)

In [7]:
import torch
from tqdm import tqdm


@torch.inference_mode()
def get_embeddings_labels(model, loader):
    """
    Функция получает на вход нейросеть и лоадер с пердобработаными постами
    Проводите обработку лоадера в режиме применения 
    На выходе получаем эмбединги для каждого поста
    """
    model.eval()
    
    total_embeddings = []
    
    for batch in tqdm(loader):
        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0)

In [8]:
#проверим возморжность работы видеокарты
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)
print(torch.cuda.get_device_name())

model = model.to(device)

cuda:0
NVIDIA GeForce GTX 1650


In [9]:
#формируем эмбединги
embeddings = get_embeddings_labels(model, loader).numpy()

embeddings

100%|████████████████████████████████████████████████████████████████████████████████| 220/220 [03:22<00:00,  1.08it/s]


array([[ 3.63150865e-01,  4.89376076e-02, -2.64081180e-01, ...,
        -1.41593322e-01,  1.59181338e-02,  9.17690195e-05],
       [ 2.36416355e-01, -1.59500718e-01, -3.27798098e-01, ...,
        -2.89936095e-01,  1.19365320e-01, -1.62343075e-03],
       [ 3.75191331e-01, -1.13944076e-01, -2.40547031e-01, ...,
        -3.38919759e-01,  5.86940572e-02, -2.12656837e-02],
       ...,
       [ 3.40382695e-01,  6.64923638e-02, -1.63184494e-01, ...,
        -8.65628794e-02,  2.03403875e-01,  3.20906416e-02],
       [ 4.32091892e-01,  1.10916262e-02, -1.17306069e-01, ...,
         7.54015967e-02,  1.02739766e-01,  1.52742090e-02],
       [ 3.04277748e-01, -7.62156770e-02, -6.77585602e-02, ...,
        -5.43488115e-02,  2.44383752e-01, -1.41487354e-02]], dtype=float32)

In [11]:
#на получены эмбединги искользуем PCA и скратим пространство до 2 измерений
from sklearn.decomposition import PCA

centered = embeddings - embeddings.mean()

pca = PCA(n_components=2,random_state = 74)

pca_decomp = pca.fit_transform(centered)

In [15]:
pd.DataFrame(pca_decomp)

,0,1
0,0.789074,1.578994
1,0.792763,1.521054
2,0.801290,1.223608
3,0.867122,0.971222
4,0.411658,0.809848
...,...,...
7018,-0.983510,-0.483879
7019,-0.821777,-0.706064
7020,-0.469401,-0.874856
7021,-1.579793,-0.448612


In [25]:

def transform_post(post, n_pca = 20, drop = True, ohe = True,lemma =True):
    """
    Функция получает на вход таблицу пост post.
    Опчионально проводит OneHotEncoding типов постов,
    Лемматизацию текстов, Tf-Idf и накладывает на полученые вектора  PCA
    
    :n_pca: кол-во изерений в новом пространстве, если 0 не проводит pca b tf-idf
    :drop: = True/False дропать ли колонку текст
    :ohe: = True/False проводить ли  OneHotEncoding колонки topic
    :lemma: =True/False проводить ли лемматизацию
    
    
    """
    post_ = post.copy()
    #wnl = WordNetLemmatizer() #закоментил что бы не качать т.к. он не нужен в этом буке
    
    #лемматизация текста
    if lemma == True:
        def preprocessing(line, token=wnl):
            line = line.lower()
            line = re.sub(r"[{}]".format(string.punctuation), " ", line)
            line = line.replace('\n\n', ' ').replace('\n', ' ')
            line = ' '.join([token.lemmatize(x) for x in line.split(' ')])
            return line
    else:
        preprocessing = None

    # учу OHE_Post
    if ohe == True:
        one_hot = pd.get_dummies(post_['topic'], prefix='topic', drop_first=True)
        post_  = pd.concat((post_.drop(['topic'], axis=1), one_hot), axis=1)
        post_transformd = post_
    elif ohe == False:
        post_transformd = post_.drop(['topic'],axis =1 )
    
    # tf-idf+ pca
    if n_pca > 0 :
        # провожу tf-idf для Post    
        tf = TfidfVectorizer(stop_words='english',
                             preprocessor=preprocessing, 
                             min_df = 5) #создаю экземпляр класса
        tf_idf_ = tf.fit_transform(post_['text'])#учу класс
        tf_idf_ = tf_idf_.toarray() - tf_idf_.mean() #центрируем данные

        list_col_pca = [f"PCA_{nn}" for nn in range(1,n_pca + 1)] 
        pca = PCA(n_components=n_pca,random_state = 74)
        
        # создаю экземплря PCA
        PCA_dataset = pca.fit_transform(tf_idf_) #провожу PCA 
        PCA_dataset = pd.DataFrame(PCA_dataset, columns=list_col_pca,index=post.index)
        
        # Трансформирую Post    
        post_transformd = pd.concat((post_transformd, PCA_dataset), axis=1)
        if drop == True:
            post_transformd = post_transformd.drop(['text'],axis=1) 
            
    
    else:
        if drop == True:
            post_transformd = post_transformd.drop(['text'],axis=1)
        
    return post_transformd

In [26]:
# таблица post без текса и раскодироваными topic
new_post_df = transform_post(posts_info,n_pca = 0, drop = True, ohe = True,lemma =False)

In [32]:
# обработаная таблица для обучения модели
new_post_df = pd.merge(new_post_df, pd.DataFrame(pca_decomp,columns = ['PCA_1', 'PCA_2']),left_index=True, right_index=True,how='left')

In [ ]:
#сохраняем данные
new_post_df.to_csv('post_transform_nn.csv', index=False) 